<a href="https://colab.research.google.com/github/collinjennings/BMDS/blob/main/extractiveSummaryDetectiveFiction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Solving Detective Fiction with LLM

I'm experimenting with treating this as an extractive summary problem. I think it's too generalized to work right now, especially considering the length of the short stories. Here's how I'm going to approach it in the first attempt:
1. Divide each story in to 500 token chunks.
2. Generate summaries of each chunk

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks'

In [ ]:
!pip install transformers -q
!pip install wandb -q

# Code for TPU packages install
#!curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [2]:
import glob
from collections import defaultdict
import re
import os
import numpy as np
import pandas as pd

In [3]:
### Chunking texts to fit T5 <512 token requirement
files = glob.glob("./data/texts/*.txt")
texts = defaultdict()
for afile in files:
      texts[re.sub('_','', afile.split('.')[0].split('/')[7])] = open(afile, encoding = 'utf-8').read()

In [4]:
for k in texts.keys():
  textList = texts[k].split()
  count = 0
  txtChunk = []
  for wrd in textList:
    txtChunk.append(wrd)
    count += 1
    if count == 500:
      texts[k] = ' '.join(txtChunk)
      txtChunk = []
      count = 0


In [ ]:
def main():
    # WandB – Initialize a new run
    wandb.init(project="transformers")

    # WandB – Config is a variable that holds and saves hyperparameters and inputs
    # Defining some key variables that will be used later on in the training
    config = wandb.config          # Initialize config
    config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
    config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
    config.TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
    config.VAL_EPOCHS = 1
    config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
    config.SEED = 42               # random seed (default: 42)
    config.MAX_LEN = 512
    config.SUMMARY_LEN = 150

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(config.SEED) # pytorch random seed
    np.random.seed(config.SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained("t5-base")


    # Importing and Pre-Processing the domain data
    # Selecting the needed columns only.
    # Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task.
    files = glob.glob("./data/texts/*.txt")
    texts = defaultdict()

    for afile in files:
      texts[re.sub('_','', afile.split('.')[0].split('/')[7])] = open(afile, encoding = 'utf-8').read()
    df = pd.read_csv('./data/BMDS_story_annotations.csv',encoding='latin-1')
    metaList =[]
    for index, row in df.iterrows():
      metaList.append({'code': row['Story Code'], 'title': row['Story Title'], 'author': row['Author Code'], 'key clue': row['Essential clue']})
    clueTextList = []
    for item in metaList:
      if item['key clue'] != '_none' and item['code'] in texts.keys():
        item['text'] = texts[item['code']]
        item['ctext'] = item['key clue']
        clueTextList.append(item)
    df = pd.DataFrame.from_dict(clueTextList)
    #df = df[['text','ctext']]
    df.ctext = 'summarize: ' + df.ctext
    print(df.head())


    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation.
    train_size = 0.8
    train_dataset=df.sample(frac=train_size,random_state = config.SEED)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)



    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    model = model.to(device)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

    # Log metrics with wandb
    wandb.watch(model, log="all")
    # Training loop
    print('Initiating Fine-Tuning for the model on our dataset')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)


    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
    print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
    for epoch in range(config.VAL_EPOCHS):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        final_df.to_csv('./models/predictions.csv')
        print('Output Files generated for review')

if __name__ == '__main__':
    main()